# Koppeling tussen was-wordt en definitietabel

Vinden we alle waarden voor rVvN, VvN en SBB uit de was-wordt lijst terug in de definitietabel? Alle vegetatietypen die niet automatisch tot H0000 leiden in de was wordt, zouden we willen terugvinden in de definietabel habitattypen.

Om deze twee tabellen goed te matchen, moeten we in ieder geval de volgende dingen doen:

- vegetatiecodes ontdoen van hoofdletters
- vegetatiecodes ontdoen van `leading zero's`
- Codes voor rompgemeenschappen en derivaatgemeenschappen in de definitietabel ontdoen van extra formatting
- De kolom `Code vegetatietype` uit de definitietabel splitsen in twee kolommen voor VvN en SBB. 

In [1]:
import re
import pandas as pd

In [2]:
ww = pd.read_excel(
    '../data/ww_cleaned.xlsx',
    dtype="string"
)

In [3]:
dt = pd.read_excel(
    '../data/dt_cleaned.xlsx', 
    dtype="string"
)

In [4]:
# Verwijderen van alle rijen uit de waswordt lijst die vertalen naar H0000 of H9999. Deze verwacht je niet terug te vinden in de definitietabel
ww = ww[ww['Habitattypen-Jakko']!='H0000']
ww = ww[ww['Habitattypen-Jakko']!='H9999']

# Verwijderen van alle rijen uit de was wordt lijst die een leeg habitattype hebben. Johannes Tonckens: "de waarde voor habitattype is denk ik alleen leeg bij de hogere niveuas (orde, verbond). Dan is het vegetatietype niet goed genoeg ontwikkeld omhet onder te bregen bij een associatie die wel naar eenhabitattype leidt. Dat het dan H0000 wordt is dus wel terecht."
ww = ww[~ww['Habitattypen-Jakko'].isna()]

# Verwijderen van alle rijen in de waswordt lijst waarin voor het habtype verwezen wordt naar subassociaties
ww = ww[~ww['Habitattypen-Jakko'].apply(lambda x: 'zie subassociaties' in x if x is not None else False)]

In [5]:
def disect_vvn(vvn_code: str):
    # klasse
    klasse = None
    try:
        klasse_span = re.search(r'^\d+', vvn_code).span()
        klasse = vvn_code[klasse_span[0]:klasse_span[1]]
    except:
        pass
        #print('klasse in VvN code not well defined')
    
    # orde en verbond
    orde = None
    try:
        orde = re.findall('[a-zA-Z]+', vvn_code)[0]
    except:
        pass
        #print('orde in VvN code not well defined')
    
    # associatie
    associatie = None
    try:
        associatie = re.findall('\d+', vvn_code)[1]
    except:
        pass
        #print('associatie in VvN code not well defined')
    
    # subassociatie
    subassociatie = None
    try:
        subassociatie = re.findall('[a-zA-Z]+', vvn_code)[1]
    except:
        pass
        #print('subassociatie in VvN code not well defined')
    
    return klasse, orde, associatie, subassociatie


def clean_vvn(vegcode: str):
    try:
        k, o, a, s = disect_vvn(vegcode)

        # verwijder leading zeros uit klasse 
        k = str(int(k))
        
        # orde en verbond naar lowercase
        o = o.lower()
        
        # verwijder leading zeros uit associatie
        if a is None:
            a = ''
        else:
            a = str(int(a))
        
        # als subassociatie niet aanwezig is, maak er een lege string van.
        if s is None:
            s = ''
        s = s.lower()
            
        return k + o + a + s
    except:
        pass

    
def get_associatie(vvn):
    k, o, a, _ = disect_vvn(vvn)
    try:
        return k + o + a
    except:
        pass
    
    
def get_verbond(vvn):
    k, o, _, _ = disect_vvn(vvn)
    try:
        return k + o
    except:
        pass

In [6]:
ww['VvN'] = ww['VvN'].apply(clean_vvn)
ww['VvN'] = ww['VvN'].astype('string')

dt['VvN'] = dt['VvN'].apply(clean_vvn)
dt['VvN'] = dt['VvN'].astype('string')

In [7]:
ww['SBB'] = ww['SBB'].str.lower()
dt['SBB'] = dt['SBB'].str.lower()

In [8]:
ww['VvN_associatie'] = ww['VvN'].apply(get_associatie)
ww['VvN_verbond'] = ww['VvN'].apply(get_verbond)

#### Overlap in VvN codes

In [9]:
vvn_ww = set(ww['VvN'][~ww['VvN'].isna()])
vvn_dt = set(dt['VvN'][~dt['VvN'].isna()])

print('{} unieke VvN codes in de was-wordt'.format(len(vvn_ww)))
print('{} unieke VvN codes in de definitietabel'.format(len(vvn_dt)))
print('{} VvN codes uit de was-wordt worden teruggevonden in de definitietabel'.format(len(vvn_ww.intersection(vvn_dt))))

428 unieke VvN codes in de was-wordt
289 unieke VvN codes in de definitietabel
269 VvN codes uit de was-wordt worden teruggevonden in de definitietabel


In [10]:
ww['VvN_in_dt'] = ww['VvN'].isin(vvn_dt)
ww['VvN_assoc_in_dt'] = ww['VvN_associatie'].isin(vvn_dt)
ww['VvN_verbond_in_dt'] = ww['VvN_verbond'].isin(vvn_dt)

dt['VvN_in_ww'] = dt['VvN'].isin(vvn_ww)

#### Overlap in SBB
Alle (op één na) waardes voor SBB uit de definitietabel worden teruggevonden in de waswordt. andersom is dat niet het geval, omdat alleen SBB-codes in de definitietabel staan waarvoor geen VvN code bestaat.

In [11]:
sbb_ww = set(ww['SBB'][~ww['SBB'].isna()])
sbb_dt = set(dt['SBB'][~dt['SBB'].isna()])

print('{} unieke SBB codes in de was-wordt'.format(len(sbb_ww)))
print('{} unieke SBB codes in de definitietabel'.format(len(sbb_dt)))
print('{} SBB codes uit de was-wordt worden teruggevonden in de definitietabel'.format(len(sbb_ww.intersection(sbb_dt))))

508 unieke SBB codes in de was-wordt
66 unieke SBB codes in de definitietabel
59 SBB codes uit de was-wordt worden teruggevonden in de definitietabel


In [12]:
sbb_dt - (sbb_ww.intersection(sbb_dt))

{'11/c', '11a-b', '26', '40a-d', '43-e', '43-f', '43-g'}

In [13]:
ww['SBB_in_dt'] = ww['SBB'].isin(sbb_dt)
dt['SBB_in_ww'] = dt['SBB'].isin(sbb_ww)

In [15]:
ww.to_excel('../data/ww_reformatted.xlsx', index=False)
dt.to_excel('../data/dt_reformatted.xlsx', index=False)

#### Eerst checken op VvN, dan associatie, dan verbond, dan SBB

Wanneer zowel VvN, VvN-associatie, VvN-verbond of SBB niet kunnen worden teruggevonden in de definitietabel, is één van de volgende dingen aan de hand:
- Habitattype doorgestreept of discussiepunt in waswordt tabel.
  - `14RG10` is doorgestreept, of tussen haakjes en discussiepunt
- Habitattype in waswordt is rood (=voorstel) en dus nog niet in deftabel
  - `42RG01`
- Waswordt geeft habitatopsomming voor associatie, terwijl er 'zie subassociaties' had moeten staan. In de defitietabel zijn alleen de subassociaties terug te vinden:
  - `17aa1`
  - `27aa2`
  - `29aa3`
  - `31ab1`
  - 
- VvN en SBB allebei niet terug te vinden:
  - `5rg3`
  - `5rg8` / `05-e`
  - `33DG2`
  - `33RG1`
  - `41Aa02(a)`